In [47]:
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip
import speech_recognition as sr
from tkinter.filedialog import askopenfilename
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import json

In [51]:
name_file = "Borja"
n_file = 1

In [48]:
# Extraigo el audio del vídeo

video = mp.VideoFileClip(".\\videos\\{}_{}.mp4".format(name_file, n_file))
audio = video.audio.write_audiofile(r".\\audios\\Borja_{}.wav".format(name_file, n_file)) # de cuando se recibia un vídeo
recibido_cambio = sr.Recognizer()
speech_audio = sr.AudioFile(".\\audios\\{}_{}.wav".format(name_file, n_file))

chunk:   0%|          | 0/1000 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in .\\audios\\Borja_1.wav


MoviePy - Done.


In [49]:
# Reconocedor básico

with speech_audio as fuente:
    #Se reduce el ruido
    recibido_cambio.adjust_for_ambient_noise(fuente)
    audio = recibido_cambio.record(fuente)
    #Transcripcion
    texto_transcrito = recibido_cambio.recognize_google(audio, language = "es-ES")
    print(texto_transcrito)

UnknownValueError: 

In [50]:
# Guardo el texto en bruto transcrito de los audios

filename_t_raw = ".\\t_raw\\{}_{}.txt".format(name_file, n_file)
with open(filename_t_raw, "w", encoding="utf-8") as f:
    f.write(texto_transcrito)
print("Se ha guardado la transcripción en el archivo " + filename_t_raw)

Se ha guardado la transcripción en el archivo .\t_raw\Borja_1.txt


In [52]:
f = open ('.\\t_processed\\{}_{}.txt'.format(name_file, n_file),'r',encoding='utf8')
text = f.read()

sw = set(stopwords.words("spanish"))
# Limpio con regex el texto
text = re.sub('[%s]' % re.escape(string.punctuation + "'" + '"' + "’" + '”' + '“' + "•‘"), ' ', str(text))
text = re.sub('\w*\d\w*', ' ', str(text))
# Pongo el texto en minúsculas
text = text.lower()
# Tokenizo por palabras
text = word_tokenize(str(text), language='spanish')
tokens = []
for t in text:
    if not t in sw:
        tokens.append(t)

json_filename = ".\\t_processed\\{}_{}.json".format(name_file, n_file)
with open(json_filename, "w", encoding='utf-8') as f:
    data = json.dump({}, f)
with open(json_filename, "r", encoding='utf-8') as f:
    data = json.load(f)
for t in tokens:
    entry = {
        'word': SnowballStemmer('spanish').stem(t),
        'start': 0.0
    }
    data.append(entry)

with open(json_filename, "w", encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)
print(data)

[{'word': 'aproxim', 'start': 0.0, 'end': 0.0}, {'word': 'rotond', 'start': 0.0, 'end': 0.0}, {'word': 'aceler', 'start': 0.0, 'end': 0.0}, {'word': 'levant', 'start': 0.0, 'end': 0.0}, {'word': 'pie', 'start': 0.0, 'end': 0.0}, {'word': 'aceler', 'start': 0.0, 'end': 0.0}, {'word': 'rotond', 'start': 0.0, 'end': 0.0}, {'word': 'medi', 'start': 0.0, 'end': 0.0}, {'word': 'rotond', 'start': 0.0, 'end': 0.0}, {'word': 'cerc', 'start': 0.0, 'end': 0.0}, {'word': 'manteg', 'start': 0.0, 'end': 0.0}, {'word': 'aceler', 'start': 0.0, 'end': 0.0}, {'word': 'izquierd', 'start': 0.0, 'end': 0.0}, {'word': 'libr', 'start': 0.0, 'end': 0.0}, {'word': 'vien', 'start': 0.0, 'end': 0.0}, {'word': 'nadi', 'start': 0.0, 'end': 0.0}, {'word': 'entro', 'start': 0.0, 'end': 0.0}, {'word': 'rotond', 'start': 0.0, 'end': 0.0}, {'word': 'gir', 'start': 0.0, 'end': 0.0}, {'word': 'izquierd', 'start': 0.0, 'end': 0.0}, {'word': 'rotond', 'start': 0.0, 'end': 0.0}, {'word': 'manteng', 'start': 0.0, 'end': 0.0}

In [73]:
instrucciones = {
    'aceler': 'ACEL-ON',
    'acerc rotond': 'RNDBT-FAR',
    'rotond medi': 'RNDBT-MED',
    'manteng aceler': 'ACEL-ON',
    'rotond cerc': 'RNDBT-NEAR',
    'izquierd libr': 'RNDBT-CLEAR',
    'ced': 'CEDA',
    'entro rotond': 'RNDBT-IN',
    'gir derech': 'TURN-R',
    'intermitent izquierd': 'BLK-L',
    'rotond gir izquierd': 'TURN-L',
    'rotond gir derech': 'TURN-R',
    'gir izquierd': 'TURN-L',
    'quit intermitent': 'BLK-OFF',
    'intermitent derech': 'BLK-R',
    'salg rotond': 'RNDBT-OUT',
    'aproxim rotond': 'RNDBT-FAR',
    'levant pie aceler': 'ACEL-OFF',
    'vien nadi': 'RNDBT-CLEAR',
    'gir izquierd rotond': 'TURN-L',
    'gir derech rotond': 'TURN-R',
    'atencion ciclist': 'WARN-VEH',
    'fren': 'BRK'
}

json_filename = ".\\t_processed\\{}_{}.json".format(name_file, n_file)
with open(json_filename, "r", encoding='utf-8') as f:
    data = json.load(f)

dataset = {}

for i, x in enumerate(data):
    w = x['word']
    if x['word'] in instrucciones:
        #print("{} => {}".format(w, instrucciones[w]))
        dataset[instrucciones[w]] = [x]
    elif (len(data) > i + 2) and x['word'] + ' ' + data[i + 1]['word'] + ' ' + data[i + 2]['word'] in instrucciones:
        sentence = x['word'] + ' ' + data[i + 1]['word'] + ' ' + data[i + 2]['word']
        dataset[instrucciones[sentence]] = [x, data[i + 1], data[i + 2]]
        #print("{} => {}".format(sentence, instrucciones[sentence]))
    elif (len(data) > i + 1) and x['word'] + ' ' + data[i + 1]['word'] in instrucciones:
        sentence = x['word'] + ' ' + data[i + 1]['word']
        dataset[instrucciones[sentence]] = [x, data[i + 1]]
        #print("{} => {}".format(sentence, instrucciones[sentence]))

json_out = ".\\t_tags\\{}_{}.json".format(name_file, n_file)
with open(json_out, "w") as f:
    json.dump(dataset, f, ensure_ascii=False)

print(dataset)

{'RNDBT-FAR': [{'word': 'aproxim', 'start': 0.0, 'end': 0.0}, {'word': 'rotond', 'start': 0.0, 'end': 0.0}], 'ACEL-ON': [{'word': 'aceler', 'start': 0.0, 'end': 0.0}], 'ACEL-OFF': [{'word': 'levant', 'start': 0.0, 'end': 0.0}, {'word': 'pie', 'start': 0.0, 'end': 0.0}, {'word': 'aceler', 'start': 0.0, 'end': 0.0}], 'RNDBT-MED': [{'word': 'rotond', 'start': 0.0, 'end': 0.0}, {'word': 'medi', 'start': 0.0, 'end': 0.0}], 'RNDBT-NEAR': [{'word': 'rotond', 'start': 0.0, 'end': 0.0}, {'word': 'cerc', 'start': 0.0, 'end': 0.0}], 'RNDBT-CLEAR': [{'word': 'vien', 'start': 0.0, 'end': 0.0}, {'word': 'nadi', 'start': 0.0, 'end': 0.0}], 'RNDBT-IN': [{'word': 'entro', 'start': 0.0, 'end': 0.0}, {'word': 'rotond', 'start': 0.0, 'end': 0.0}], 'TURN-L': [{'word': 'gir', 'start': 0.0, 'end': 0.0}, {'word': 'izquierd', 'start': 0.0, 'end': 0.0}], 'WARN-VEH': [{'word': 'atencion', 'start': 0.0, 'end': 0.0}, {'word': 'ciclist', 'start': 0.0, 'end': 0.0}], 'BRK': [{'word': 'fren', 'start': 0.0, 'end': 0.0}